In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # mean and std for MNIST
])

# Load training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:40<00:00, 247443.87it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 122036.69it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:06<00:00, 257986.21it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 529063.78it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [44]:
import torch.nn as nn
import torch.nn.functional as F

class mlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 10)
        )
        self.criterion = F.cross_entropy

    def forward(self, x):
        x = x.view(x.shape[0], 28*28)
        logits = self.seq(x) # 64, 10
        probs = F.softmax(logits, dim=1) # 64 , 10
        return probs # (64, 10) tensor

In [43]:
for [x, y] in train_loader:
    print(x.shape[0])
    print(y)
    break

64
tensor([0, 7, 5, 9, 4, 8, 1, 0, 2, 6, 8, 1, 9, 3, 6, 3, 0, 0, 8, 8, 7, 1, 7, 2,
        5, 6, 6, 8, 5, 1, 7, 0, 0, 5, 4, 6, 0, 7, 2, 1, 6, 1, 6, 9, 4, 6, 0, 5,
        8, 0, 6, 0, 1, 6, 5, 8, 7, 1, 1, 2, 9, 1, 0, 7])


In [ ]:
# training loop
import numpy as np
train_losses = []
test_losses = []
model = mlp()
optimizer = torch.optim.Adam(lr = 5e-4 , betas = (0.90, 0.99), params=model.parameters())
num_epochs = 10
for epoch in range(num_epochs):
    print("epoch", epoch)
    total_train_loss = 0
    total_test_loss = 0
    for [x, y] in train_loader:
        optimizer.zero_grad()
        probs = model(x)
        loss = model.criterion(probs, y)
        loss.backward()
        optimizer.step()
        total_train_loss += torch.sum(loss).item()
    train_losses.append(total_train_loss / len(train_loader))
    with torch.no_grad():
        for [x,y] in test_loader:
            probs = model(x)
            loss = model.criterion(probs, y)
            total_test_loss += torch.sum(loss).item()
        test_losses.append(total_test_loss / len(test_loader))

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7


In [ ]:
import matplotlib.pyplot as plt

x = list(np.arange(num_epochs))
plt.plot(x, train_losses)
plt.plot(x, test_losses)
plt.show()